In [0]:
%pip install faker

from faker import Faker
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import current_timestamp, from_utc_timestamp
import pandas as pd

spark.sql("TRUNCATE TABLE data_catalog_01_d.bronze.clientes")

fake = Faker('pt_BR')
data = []
for i in range(10000):
    data.append({
        "id": i + 1,
        "cnpj": fake.cnpj(),
        "razao_social": fake.company(),
        "fantasia": fake.company_suffix(),
        "cidade": fake.city(),
        "endereco": fake.street_name(),
        "numero": fake.building_number(),
        "estado": fake.estado_sigla(),
        "vendedor": fake.name(),
    })

df_pd = pd.DataFrame(data)
df_spark = spark.createDataFrame(df_pd)
df_spark = df_spark.withColumn("data_carga",from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"))
df_spark.write.format("delta").mode("append").saveAsTable("data_catalog_01_d.bronze.clientes")

In [0]:
# Adiciona 15 novos registros à tabela
novos_dados = []
for i in range(2):
    novos_dados.append({
        "id": None,  # Será ajustado abaixo
        "cnpj": fake.cnpj(),
        "razao_social": fake.company(),
        "fantasia": fake.company_suffix(),
        "cidade": fake.city(),
        "endereco": fake.street_name(),
        "numero": fake.building_number(),
        "estado": fake.estado_sigla(),
        "vendedor": fake.name(),
    })

# Descobre o maior ID atual para continuar a sequência
max_id = spark.sql("SELECT MAX(id) as max_id FROM data_catalog_01_d.bronze.clientes").collect()[0]['max_id'] or 0
for idx, row in enumerate(novos_dados):
    row['id'] = max_id + idx + 1

df_novos = pd.DataFrame(novos_dados)
df_novos_spark = spark.createDataFrame(df_novos)
df_novos_spark = df_novos_spark.withColumn("data_carga",from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"))
df_novos_spark.write.format("delta").mode("append").saveAsTable("data_catalog_01_d.bronze.clientes")


In [0]:
# Seleciona aleatoriamente alguns IDs existentes para recarregar com novo nome de vendedor
from pyspark.sql.functions import rand

ids_aleatorios = (
    spark.table("data_catalog_01_d.bronze.clientes")
    .orderBy(rand())
    .limit(5)
    .select("id")
    .toPandas()["id"].tolist()
)

# Para cada ID selecionado, recarrega o registro com novo nome de vendedor e atualiza data_carga
from pyspark.sql.functions import lit

for id_alterar in ids_aleatorios:
    registro = (
        spark.table("data_catalog_01_d.bronze.clientes")
        .filter(f"id = {id_alterar}")
        .orderBy("data_carga", ascending=False)
        .limit(1)
    )
    registro = registro.withColumn("vendedor", lit(fake.name()))
    registro = registro.withColumn("data_carga",from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"))
    registro.write.format("delta").mode("append").saveAsTable("data_catalog_01_d.bronze.clientes")